In [15]:
%load_ext autoreload
%autoreload 2
import torch
import h5py as h5
import matplotlib.pyplot as plt
import os
import configparser
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import sys
import optuna
from sklearn.model_selection import ParameterGrid
#sys.path.append('/projects/EKOLEMEN/survival_tm/train_models/auton-survival')
sys.path.append(os.path.expanduser("~/TMPredictor/survival_tm/auton-survival"))
import os
auton_path = os.path.expanduser("~/TMPredictor/survival_tm/auton-survival")
os.environ["PYTHONPATH"] = f"{auton_path}:" + os.environ.get("PYTHONPATH", "")

from sklearn.model_selection import ParameterGrid
from auton_survival.estimators import SurvivalModel
from auton_survival.metrics import survival_regression_metric
from auton_survival.models.dsm import DeepSurvivalMachines
from sksurv.metrics import concordance_index_ipcw, brier_score, cumulative_dynamic_auc
from get_survival_from_shot import get_rt_survival_from_shot, get_cakenn_survival_from_shot
import metrics_helpers
import plotting_helpers
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
from mpmath.functions.zetazeros import search_supergood_block

import metrics_helpers
from functools import partial
import ast
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune.progress_reporter import CLIReporter
from ray.train import report

In [17]:
from ray.tune import ExperimentAnalysis
import os
import pandas as pd
from collections import Counter

# Load analysis object
analysis = ExperimentAnalysis(os.path.expanduser('~/ray_results/hyperparams_25'))

# Best trial
best_trial = analysis.get_best_trial(metric="val_score", mode="max")
print("✅ Best Trial ID:", best_trial.trial_id)
print("📋 Best Trial Config:", best_trial.config)
print("📈 Best Trial Last Result:", best_trial.last_result)

# Total trials
print(f"\n🔢 Total trials: {len(analysis.trials)}")

# Trial status breakdown
statuses = Counter([trial.status for trial in analysis.trials])
print("📊 Trial statuses:", dict(statuses))

# Convert all results to DataFrame
df = analysis.dataframe(metric="val_score", mode="max")
pd.set_option("display.max_columns", None)  # Show all columns
print("\n📄 Full Trials DataFrame:")
# Show available columns
print("Available columns:\n", df.columns.tolist())

columns_to_show = [
    "trial_id", "train_loss", "val_loss", "train_score", "val_score",
    "training_iteration", "config/k", "config/layers", "config/neurons",
    "config/learning_rate", "config/batch_size", "config/dropout", "time_this_iter_s"
]
if "config/distribution" in df.columns:
    columns_to_show.append("config/distribution")
if "config/iters" in df.columns:
    columns_to_show.append("config/iters")
if "config/seed" in df.columns:
    columns_to_show.append("config/seed")

# Sort by best validation score
df_sorted = df[columns_to_show].sort_values("val_score", ascending=False)

# Set float formatting
pd.set_option("display.float_format", lambda x: f"{x:0.10f}")
# display all rows
pd.set_option("display.max_rows", None)
df_sorted.head(105)

✅ Best Trial ID: 167f0_00049
📋 Best Trial Config: {'iters': 50, 'k': 2, 'layers': 4, 'neurons': 128, 'learning_rate': 0.00017206480468394527, 'batch_size': 256, 'dropout': 0.42635581869662226, 'distribution': 'LogNormal', 'seed': 723}
📈 Best Trial Last Result: {'train_loss': 807.1746959510845, 'val_loss': 0.49933051719169, 'train_score': 0.8858505840937625, 'val_score': 0.839756795411449, 'val_median_warning_time': nan, 'timestamp': 1750954127, 'checkpoint_dir_name': None, 'done': True, 'training_iteration': 1, 'trial_id': '167f0_00049', 'date': '2025-06-26_12-08-47', 'time_this_iter_s': 848.8338375091553, 'time_total_s': 848.8338375091553, 'pid': 2760334, 'hostname': 'stellar-i04n6', 'node_ip': '10.33.81.69', 'config': {'iters': 50, 'k': 2, 'layers': 4, 'neurons': 128, 'learning_rate': 0.00017206480468394527, 'batch_size': 256, 'dropout': 0.42635581869662226, 'distribution': 'LogNormal', 'seed': 723}, 'time_since_restore': 848.8338375091553, 'iterations_since_restore': 1, 'experiment_

,trial_id,train_loss,val_loss,train_score,val_score,training_iteration,config/k,config/layers,config/neurons,config/learning_rate,config/batch_size,config/dropout,time_this_iter_s,config/distribution,config/iters,config/seed
49,167f0_00049,807.1746959511,0.4993305172,0.8858505841,0.8397567954,1,2,4,128,0.0001720648,256,0.4263558187,848.8338375092,LogNormal,50,723
734,167f0_00734,480.6610925574,0.4543382258,0.8440698244,0.8334572497,1,2,2,128,0.0000554873,512,0.1978023277,474.0409278870,LogNormal,50,436
950,167f0_00950,444.8143226956,0.5568837119,0.8528200013,0.8305153693,1,3,2,64,0.0008050290,512,0.1614883690,475.7504310608,LogNormal,50,19
843,167f0_00843,441.8206633515,0.5719602718,0.8449334529,0.8304205503,1,2,3,64,0.0005642681,512,0.4917411734,477.8634788990,LogNormal,50,797
651,167f0_00651,1609.8708219521,0.5124957621,0.8747553136,0.8302196284,1,4,3,256,0.0000330225,128,0.4196337643,1431.0237903595,LogNormal,50,393
686,167f0_00686,1578.4185050848,0.5699253685,0.8443231007,0.8299771366,1,2,2,128,0.0005732756,128,0.1873301839,904.9701795578,LogNormal,50,19
490,167f0_00490,912.8486247445,0.4537351148,0.8511452997,0.8296155810,1,4,4,64,0.0002468457,256,0.2596560295,744.5818631649,LogNormal,50,408
827,167f0_00827,1599.9756376561,0.4810462715,0.8611312048,0.8292550975,1,4,4,128,0.0003410419,128,0.3848512067,1298.1709392071,LogNormal,50,436
841,167f0_00841,370.3027576708,0.5342885845,0.8600378308,0.8260407838,1,2,3,256,0.0001227932,512,0.0949063039,599.6571257114,LogNormal,50,127
953,167f0_00953,869.8595995277,0.5077607078,0.8204932506,0.8256640094,1,4,1,256,0.0007987694,256,0.6365131519,659.7041931152,LogNormal,50,436
